## Query data

In [2]:
import os
from os.path import (
    abspath,
    dirname,
)
from prometheus_api_client import (
    PrometheusConnect,
    MetricSnapshotDataFrame,
    MetricRangeDataFrame
)
import sys
import datetime as dt
import pandas as pd
import pytz
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [3]:
prom = PrometheusConnect(url="http://192.168.50.113:19090", disable_ssl=True)

query_info_df = pd.read_csv("query_info.csv")

# read text file
with open("query.txt") as f:
    query_list = f.readlines()

interval = "1m"
# replace substring of each line in query_list
query_list = list(map(lambda x: x.replace(
    '$host', '10.244.2.13:9104'), query_list))
query_list = list(map(lambda x: x.replace('$interval', interval), query_list))
query_save_path = os.path.join(
    dirname(dirname(os.getcwd())), "data", "mysql", "test1", interval)

dropped = [
    # "Top_Command_Counters.csv", "MySQL_Handlers.csv",
    "MySQL_Table_Open_Cache_Status__Table_Open_Cache_Hit_Ratio.csv",
    "MySQL_Connections__Max_Connections.csv",
    'MySQL_Internal_Memory_Overview__InnoDB_Log_Buffer_Size.csv',
    'MySQL_Internal_Memory_Overview__Key_Buffer_Size.csv',
    'MySQL_Internal_Memory_Overview__Query_Cache_Size.csv',
    'MySQL_Open_Files__Open_Files_Limit.csv',
    'MySQL_Open_Tables__Table_Open_Cache.csv',
    'MySQL_Query_Cache_Memory__Query_Cache_Size.csv',
    'MySQL_Table_Definition_Cache__Table_Definitions_Cache_Size.csv',
    'MySQL_Thread_Cache__Thread_Cache_Size.csv',
    # 'MySQL_Query_Duration.csv',
    # 'System_IO_Activity__Page_In.csv',
    # 'System_IO_Activity__Page_Out.csv',
    # 'System_Memory_Distribution__Free.csv',
    # 'System_Memory_Distribution__Total.csv',
    # 'System_CPU_Usage_Load.csv',
    # 'System_CPU_Usage_Load__Max_Core_Utilization.csv',
    # 'System_CPU_Usage_Load__Load_1m.csv',
    # 'System_Network_Traffic__Inbound.csv',
    # 'System_Network_Traffic__Outbound.csv',
    # 'System_Swap_Activity__Swap_In_Reads.csv',
    # 'System_Swap_Activity__Swap_Out_Writes.csv',
    # 'System_Disk_Latency.csv',
    'Pod_User_CPU_Usage.csv',
    'Pod_System_CPU_Usage.csv',
    'Pod_Memory_Usage.csv',
    'Pod_Memory_Failures.csv',
    'Pod_Disk_Writes.csv',
    'Pod_Disk_Reads.csv',
    'Pod_Network_Received.csv',
    'Pod_Network_Transmitted.csv',
]


In [9]:
# set query time
# start_time = dt.datetime(2022, 5, 21, 1, 00, 00)+dt.timedelta(hours=8)
# start_time = dt.datetime(2022, 5, 27, 19, 30, 00)+dt.timedelta(hours=8)
start_time = dt.datetime(2022, 5, 29, 21, 10, 00)+dt.timedelta(hours=8)
# start_time = end_time-dt.timedelta(hours=6)

In [10]:
for i in range(len(query_list)):
    query_df = pd.DataFrame()
    for t in range(1):
        mstart_time = start_time+dt.timedelta(hours=1)*t
        mend_time = mstart_time+dt.timedelta(hours=1)-dt.timedelta(seconds=1)
        metric_data = prom.custom_query_range(
            query_list[i],
            start_time=mstart_time,
            end_time=mend_time,
            step=1
        )

        metric_df = pd.DataFrame(columns=['timestamp'])
        for m in metric_data:
            if query_info_df.metric[i] == 'None':
                col_name = "None"
            else:
                if m['metric'] == {} and len(metric_data) > 1:
                    continue
                sub_metrics = query_info_df.metric[i].split("+")
                col_name = "_".join(
                    list(map(lambda x: m['metric'][x], sub_metrics)))
                col_name = col_name.replace('-', '_')
            temp_df = pd.DataFrame(m['values'], columns=[
                                   'timestamp', col_name])
            metric_df = pd.merge(metric_df, temp_df,
                                 on='timestamp', how='outer')

        query_df = pd.concat([query_df, metric_df])

    save_path = os.path.join(query_save_path, '_'.join(
        (query_info_df.name[i]).split())+'.csv')
    query_df.to_csv(save_path, index=False)


## Combine data

In [4]:
combined_df = pd.DataFrame(columns=['timestamp'])
# walk through directory
for root, dirs, files in os.walk(query_save_path):
    for file in files:
        if file.endswith(".csv"):
            if file not in dropped:
                temp_df = pd.read_csv(os.path.join(root, file))
                if temp_df.empty:
                    continue
                file_name = file.split('.')[0]
                # rename temp_df columns
                temp_df.columns = ['timestamp'] + list(
                    map(lambda x: file_name+'__'+x, temp_df.columns[1:]))
                combined_df = pd.merge(combined_df, temp_df,
                                    on='timestamp', how='outer')
combined_df.set_index('timestamp')
# drop columns all nan
combined_df.dropna(axis=1, how='all', inplace=True)
combined_df.interpolate(inplace=True, limit=60, method='linear')
# fill nan by 0
combined_df.fillna(0, inplace=True)
combined_df = combined_df.loc[:, (combined_df != 0).any(axis=0)]

combined_df.to_csv(os.path.join(
    dirname(dirname(os.getcwd())), "output", "mysql", 'test1_'+interval+".csv"), index=False)


In [5]:
temp_df = combined_df.copy()
temp_df.drop(['timestamp'], axis=1, inplace=True)
# minus by mean
temp_df = temp_df - temp_df.mean()
blip_df = pd.DataFrame()
# chaos_state_df = pd.read_csv(os.path.join(
#     os.pardir, "output", 'chaos_state.csv'))
# blip_df["chaos_state"]=chaos_state_df["chaos_state"]

# discritize each single column by kmeans cluster
for col in temp_df.columns:
    # if col=="chaos__event_type":
    #     blip_df[col]=temp_df[col]
    #     continue
    kmeans = KMeans(n_clusters=10, random_state=0)
    kmeans.fit(temp_df[col].to_numpy().reshape(-1, 1))
    blip_df[col] = kmeans.labels_


blip_df.to_csv(os.path.join(dirname(dirname(os.getcwd())),
               "output", "mysql", 'train_blip_'+interval+".csv"), index=False)


C:\Users\jzlch\AppData\Local\Temp/ipykernel_12364/1165669979.py:16: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans.fit(temp_df[col].to_numpy().reshape(-1, 1))
C:\Users\jzlch\AppData\Local\Temp/ipykernel_12364/1165669979.py:16: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans.fit(temp_df[col].to_numpy().reshape(-1, 1))
C:\Users\jzlch\AppData\Local\Temp/ipykernel_12364/1165669979.py:16: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans.fit(temp_df[col].to_numpy().reshape(-1, 1))
C:\Users\jzlch\AppData\Local\Temp/ipykernel_12364/1165669979.py:16: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans.fit(temp_df[col].to_numpy().reshape(-1, 1))
C:\Users

In [ ]:
# copy combined_df
notears_raw_df = combined_df.copy()
# standard normalization
notears_raw_df = (notears_raw_df - notears_raw_df.mean()) / \
    (notears_raw_df.std())
notears_raw_df.dropna(axis=1, how='all', inplace=True)
notears_raw_df.to_csv(os.path.join(os.pardir, "output",
                      'notears_raw.csv'), index=False)

notears_df = notears_raw_df.drop(['timestamp'], axis=1)
notears_df.to_csv(os.path.join(os.pardir, "output",
                  'notears.csv'), index=False, header=False)


In [4]:
offline_df=pd.read_csv(r"C:\Users\jzlch\Documents\Project\TiDB_exp\output\mysql\train_1m_plus.csv")
online_df=pd.read_csv(r"C:\Users\jzlch\Documents\Project\TiDB_exp\output\mysql\test_1m.csv")

In [6]:
blip_offline_df = pd.DataFrame()
blip_online_df = pd.DataFrame()
for col in offline_df.columns[1:]:
    kmeans = KMeans(n_clusters=10, random_state=0)
    kmeans.fit(offline_df[col].to_numpy().reshape(-1, 1))
    blip_offline_df[col] = kmeans.labels_
    if col in online_df.columns:
        kmeans.fit(online_df[col].to_numpy().reshape(-1, 1))
        blip_online_df[col] = kmeans.labels_
    else:
        blip_online_df[col] = 0


C:\Users\jzlch\AppData\Local\Temp/ipykernel_17156/1889268263.py:5: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans.fit(offline_df[col].to_numpy().reshape(-1, 1))
C:\Users\jzlch\AppData\Local\Temp/ipykernel_17156/1889268263.py:8: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans.fit(online_df[col].to_numpy().reshape(-1, 1))
C:\Users\jzlch\AppData\Local\Temp/ipykernel_17156/1889268263.py:5: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans.fit(offline_df[col].to_numpy().reshape(-1, 1))
C:\Users\jzlch\AppData\Local\Temp/ipykernel_17156/1889268263.py:8: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  kmeans.fit(online_df[col].to_numpy().reshape(-1, 1))
C:

In [11]:
blip_offline_df.to_csv(os.path.join(dirname(dirname(os.getcwd())),
               "output", "mysql", "offline.csv"), index=False)
blip_online_df.to_csv(os.path.join(dirname(dirname(os.getcwd())),
                "output", "mysql", "online.csv"), index=False)

blip_offline_less_df=blip_offline_df.iloc[37800:,:]
blip_offline_less_df.to_csv(os.path.join(dirname(dirname(os.getcwd())),
               "output", "mysql", "offline_withoutChaos.csv"), index=False)

In [ ]:
np.where(pd.isnull(new_df))

In [ ]:
df=pd.read_csv(r"C:\Users\jzlch\Documents\Project\TiDB_exp\data\mysql\train\1s\MySQL_Client_Thread_Activity__Peak_Threads_Connected.csv")

In [ ]:
df.shape

In [ ]:
df.plot(x=df.columns[0], y=df.columns[1:])